In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, split, col, size, array_contains
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd


In [2]:
spark = SparkSession.builder \
        .appName("Silver") \
        .master("local[*]") \
        .getOrCreate()

#### Read CSV file

In [23]:
bus_facility = spark.read.csv("../raw/bus_facilities.csv", header=True)

bus_facility.show(truncate=False)

+---+---------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |Bus_Name             |Facilities                                                                                                                                                                                                                                              |
+---+---------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |Mạnh Hùng            |['Đèn đọc sách', 'Dép', 'Dây đai an toàn', 'Gửi kèm xe máy', 'Nước uống', 'Gối nằm', 'Búa phá kính', 'Tivi LED', 'Sạc điện thoại', 'Rèm cửa', 

In [24]:
bus_facility.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Bus_Name: string (nullable = true)
 |-- Facilities: string (nullable = true)



#### Regex column Facilities

In [25]:
# Loại bỏ ký tự [ và ] trong cột Facilities
bus_facility = bus_facility.withColumn("Facilities", regexp_replace(col("Facilities"), "[\\[\\]']", ""))

# Chuyển STRING thành ARRAY bằng cách tách theo dấu phẩy và khoảng trắng
bus_facility = bus_facility.withColumn("Facilities", split(col("Facilities"), ", "))

# Remove rows where the Facilities array is empty or contains only empty strings
bus_facility = bus_facility.filter((size(col("Facilities")) > 0) & (~array_contains(col("Facilities"), "")))


bus_facility.show(truncate=False)

bus_facility.printSchema()


+---+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |Bus_Name             |Facilities                                                                                                                                                                                                            |
+---+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |Mạnh Hùng            |[Đèn đọc sách, Dép, Dây đai an toàn, Gửi kèm xe máy, Nước uống, Gối nằm, Búa phá kính, Tivi LED, Sạc điện thoại, Rèm cửa, Dàn âm thanh (Loa), Chăn đắp, Wifi, Điều hòa]                                               |
|2  |Toàn Khải            |[

#### Read bus id company file

In [26]:
bus_ids_company = spark.read.csv("../raw/bus_ids.csv", header=True)

bus_ids_company.show(20)

+--------------------+--------------------+----------+
|              Bus_Id|            Bus_Name|Company_ID|
+--------------------+--------------------+----------+
|vu-linh-limousine...|   Vũ Linh limousine|       768|
|       ba-chau_13078|             Ba Châu|     13078|
|      van-lang_11826|            Văn Lang|     11826|
|      tri-nhan_12911|            Trí Nhân|     12911|
|     tuan-hiep_11899|           Tuấn Hiệp|     11899|
|thien-thanh-limou...|Thiện Thành Limou...|     21843|
|           hao_12797|                 Hảo|     12797|
|tri-nhan-limousin...|  Trí Nhân Limousine|     12911|
|         ha-my_12507|               Hà My|     12507|
|     toan-khai_46289|           Toàn Khải|     46289|
|     giap-diep_12772|           Giáp Diệp|     12772|
|       tu-tien_13184|             Tư Tiến|     13184|
|     manh-hung_11188|           Mạnh Hùng|     11188|
|     ngoc-diem_40437|           Ngọc Diễm|     40437|
|      tan-nien_12462|            Tân Niên|     12462|
|duy-thao-

#### Join 2 tables and pick column Bus_Id

In [27]:
from pyspark.sql.functions import col

def add_bus_id(bus_facility, bus_ids_company, join_column="Bus_Name"):
    bus_facility_with_id = bus_facility.join(
        bus_ids_company.select(col(join_column), col("Bus_Id")),
        on=join_column,
        how="left"
    )
    
    # Sắp xếp lại thứ tự cột
    columns_order = ["Id", "Bus_Id" , "Bus_Name"] + [col for col in bus_facility_with_id.columns if col not in ["Id", "Bus_Id" , "Bus_Name"]]
    bus_facility_with_id = bus_facility_with_id.select(*columns_order)
    
    return bus_facility_with_id

# Gọi hàm để thêm cột bus_id và sắp xếp lại thứ tự cột
bus_facility = add_bus_id(bus_facility, bus_ids_company)

In [28]:
bus_facility.show(20, truncate=False)

+---+-------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |Bus_Id                   |Bus_Name             |Facilities                                                                                                                                                                                                            |
+---+-------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |manh-hung_11188          |Mạnh Hùng            |[Đèn đọc sách, Dép, Dây đai an toàn, Gửi kèm xe máy, Nước uống, Gối nằm, Búa phá kính, Tivi LED, Sạc điện thoại, Rèm cửa, Dàn âm thanh (Loa)

#### Table Facility_Name

In [29]:
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

# Tách các giá trị duy nhất từ cột Facilities
facility_names = (
    bus_facility
    .select(F.explode("Facilities").alias("Facility_Name"))  # Tách từng giá trị
    .distinct()  # Loại bỏ trùng lặp
    .orderBy("Facility_Name")  # Sắp xếp theo thứ tự
)

# Gán Facility_Id theo thứ tự từ 1 trở đi
facility_names = facility_names.withColumn(
    "Facility_Id", F.row_number().over(Window.orderBy("Facility_Name"))
)

# Hiển thị bảng Facility_Names
facility_names.select("Facility_Id", "Facility_Name").show()

+-----------+--------------------+
|Facility_Id|       Facility_Name|
+-----------+--------------------+
|          1|        Búa phá kính|
|          2|            Chăn đắp|
|          3|  Dàn âm thanh (Loa)|
|          4|     Dây đai an toàn|
|          5|                 Dép|
|          6|         Ghế massage|
|          7|             Gối nằm|
|          8|              Gối ôm|
|          9|      Gửi kèm xe máy|
|         10|           Khăn lạnh|
|         11|Nhân viên sử dụng...|
|         12|           Nước uống|
|         13|             Rèm cửa|
|         14|      Sạc điện thoại|
|         15|            Tai nghe|
|         16|            Tivi LED|
|         17|              Toilet|
|         18|                Wifi|
|         19|     Xe trung chuyển|
|         20|            Điều hòa|
+-----------+--------------------+
only showing top 20 rows



#### Table Facility

In [34]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Tách từng giá trị Facilities của mỗi Bus_Id
bus_facilities_exploded = bus_facility.select(
    "Bus_Id","Bus_Name", F.explode("Facilities").alias("Facility_Name")
)

# Gán Facility_Id cho mỗi Bus_Id
bus_facilities_with_id = bus_facilities_exploded.join(
    facility_names, on="Facility_Name", how="inner"
).select("Bus_Id","Bus_Name", "Facility_Id").distinct()

# Hiển thị kết quả
bus_facilities_with_id.orderBy("Bus_Name", "Facility_Id").show(50)


+--------------------+-------------------+-----------+
|              Bus_Id|           Bus_Name|Facility_Id|
+--------------------+-------------------+-----------+
|   an-hoa-hiep_11720|        An Hoà Hiệp|          1|
|   an-hoa-hiep_11720|        An Hoà Hiệp|          2|
|   an-hoa-hiep_11720|        An Hoà Hiệp|          3|
|   an-hoa-hiep_11720|        An Hoà Hiệp|          4|
|   an-hoa-hiep_11720|        An Hoà Hiệp|          5|
|   an-hoa-hiep_11720|        An Hoà Hiệp|          7|
|   an-hoa-hiep_11720|        An Hoà Hiệp|          9|
|   an-hoa-hiep_11720|        An Hoà Hiệp|         10|
|   an-hoa-hiep_11720|        An Hoà Hiệp|         12|
|   an-hoa-hiep_11720|        An Hoà Hiệp|         13|
|   an-hoa-hiep_11720|        An Hoà Hiệp|         19|
|   an-hoa-hiep_11720|        An Hoà Hiệp|         20|
|     anh-chinh_49740|          Anh Chính|          2|
|     anh-chinh_49740|          Anh Chính|          4|
|     anh-chinh_49740|          Anh Chính|          7|
|     anh-